In [1]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [2]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'na'  # euw1 na1 kr oc1
LEAGUE='challengers'
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344

In [3]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [4]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,Y99KBNJIEqW0rblSMeJibH4iM0DMZccMqJEYaF83XNuMcKg,rng4L_9quiDxiRVNK-HzbTsERX5HeKQXlIdPHYvGrIvItjg,hsuYXDf4eJrysAn9D2h-r5-Ze-rGGGh8YSoAXr7xoHsdSC...,a hickey,5091,1656514110000,282,Y99KBNJIEqW0rblSMeJibH4iM0DMZccMqJEYaF83XNuMcKg,a hickey,899,I,140,107,False,False,True,False
1,oXGAbQmOTEpCSwQYAthmK5nPs7bcgLdAgFZeh1l3ugE1tlc,t-5dsnJRTIugo5CMfYT-KGtIKF9NyA7CvnG4ft44vn-9vc...,SDTrH9jwmVh_s-Qa-9CuQB7D9tcj5GEso9dPyPH300SnfF...,엠브로돈까스,5184,1656455475000,86,oXGAbQmOTEpCSwQYAthmK5nPs7bcgLdAgFZeh1l3ugE1tlc,엠브로돈까스,742,I,153,126,False,False,False,True
2,cZeJbdlkfxV-1hAPCKy9_BNgHA98H3llM-OilKZEV42ZJg,ysFQPR6dGBbKW_scWrpOYqV5kbXmbZEmFqVI37gwPMZP,hCk2s22f6le2_m2kshmcVvnmJ5Iop0yZxvleqKdwNFJUDT...,KRThreshNO1,1297,1656510113000,259,cZeJbdlkfxV-1hAPCKy9_BNgHA98H3llM-OilKZEV42ZJg,KRThreshNO1,500,I,94,64,False,False,True,False
3,QSFX2-SwZpfjemIFpWSyPaqxOTC5vjUJnx7qVTlgNRVKhA,XRFue3IcNfu8uNPIYdNDtmAiAjfrEGRmEMhKPVHs7Yg,3aVusfNsyRnDGoToWzO9d4VBRCueHxzSxvgx6FTG8a3C3Y...,광어콩,4369,1656454916000,180,QSFX2-SwZpfjemIFpWSyPaqxOTC5vjUJnx7qVTlgNRVKhA,광어콩,544,I,127,99,False,False,True,False
4,3le0r_oEWvCEfFR0E5qll5R7_VNnVe4LiZXou2zVDQ1rpOg,w13IEOET3s9xW1GoYEbnWcHS7EF1pQnsBmvelGvijRFCdY0,hWr0xqH6cRutVgl23qV-MUlCsXH6KVvG2xrSqpJ1bdXSfK...,uL Pado,4498,1656492724850,200,3le0r_oEWvCEfFR0E5qll5R7_VNnVe4LiZXou2zVDQ1rpOg,uL Pado,821,I,153,91,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,TK-ooWt78ZkPRLBgmUurADYT5p8ErJTS0D7x4GFmjrfJTw,qGfkqUZkS4dpCP_R93uuVabWbRyFcIe9t2Brn6KhuU7M,pGkp56l8RgEzJT7HHaSFubmSSfyLESaZbUvTAqpYrpcOA6...,개소리 한소리,3890,1656398347000,260,TK-ooWt78ZkPRLBgmUurADYT5p8ErJTS0D7x4GFmjrfJTw,개소리 한소리,525,I,106,62,False,False,True,False
133,RSHUV2CCH09oAxjixUvp7ysitX0D7Mf5fp4Vj7J6PrfXFA,31ZoAaPUcxYtXNUcJToW-B8bUa3sPb6DWyNWrWtea7KX,WvriYb865I-mtv3xZ-Gf96VIgf-JTxqucrfjYSK_ApT7zi...,Gaross,7,1655965726000,23,RSHUV2CCH09oAxjixUvp7ysitX0D7Mf5fp4Vj7J6PrfXFA,Gaross,510,I,81,47,False,False,True,True
134,P1kGfyCYYGJd5A0VcdXq4gbZRRZMd9K3uMij2cmVgMcWmv...,M8fge8Dg4AA1XvVQYgxTb8uftqjoHWKX4Exm3wdHqsqzYO...,ch9_8jtliwQErSOVEyKtGieSBchGIQthY95Lq8RVGHRqn6...,incheonfist,3546,1656510157739,9,P1kGfyCYYGJd5A0VcdXq4gbZRRZMd9K3uMij2cmVgMcWmv...,incheonfist,651,I,97,67,False,False,False,False
135,i-TwGIQCud3U3s04BSoSrMsRiE1cR76xNjMlg4wcbri0TbY,H2Jl9mkLUj_LHuqD4xOta4Gufnc_C-wxOemYSw8yu8frT6...,elsM74BtOwlhbNnsrCW2yW9wy3dXf82iYUP_FjIxtkwlkr...,칼픽 다리,5331,1656485832336,232,i-TwGIQCud3U3s04BSoSrMsRiE1cR76xNjMlg4wcbri0TbY,칼픽 다리,682,I,101,50,False,False,False,False


In [5]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [6]:
len(uniq_matches_id)

2339

In [7]:
len(matches_asset)

3423

In [8]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [9]:
len(uniq_matches)

2339

In [10]:
matches_asset[300]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [11]:
latest_matches = [match for match in uniq_matches if LATEST_RELEASE in match['info']['game_version']]

In [12]:
len(latest_matches)

1658

In [13]:
# latest_matches[0]['info']

In [14]:
matches_array = []

for match_row in latest_matches:
    match_id = match_row['metadata']['match_id']

    for participant in match_row['info']['participants']:
        match = {}
        match['match_id'] = match_id
        # match['level'] = participant['level']
        match['placement'] = participant['placement']
        # match['players_eliminated'] = participant['players_eliminated']
        # match['total_damage_to_players'] = participant['total_damage_to_players']

        for augment_index, augment in enumerate(participant['augments']):
            if augment == 'TFT7_Augment_GuildLootHR':
                augment = 'TFT7_Augment_BandOfThieves1'
            match[f'augment{augment_index}'] = augment

        for trait_index, trait in enumerate(participant['traits']):
            match[f'{trait["name"]}'] = trait["tier_current"]

        for unit_index, unit in enumerate(participant['units']):
            match[f'{unit["character_id"]}'] = unit["tier"]
            for item_index, item in enumerate(unit['itemNames']):
                match[f'{unit["character_id"]}_item{item_index}'] = item

        matches_array.append(match)

In [15]:
matches_array[0]
# len(matches_array)

{'match_id': 'KR_5993867535',
 'placement': 7,
 'augment0': 'TFT6_Augment_SalvageBin',
 'augment1': 'TFT6_Augment_RadiantRelics',
 'augment2': 'TFT6_Augment_ComponentGrabBag',
 'Set7_Dragon': 1,
 'Set7_Evoker': 0,
 'Set7_Jade': 2,
 'Set7_Legend': 0,
 'Set7_Mystic': 0,
 'Set7_Shapeshifter': 1,
 'TFT7_Gnar': 2,
 'TFT7_Gnar_item0': 'TFT_Item_ThiefsGloves',
 'TFT7_Gnar_item1': 'TFT_Item_NegatronCloak',
 'TFT7_Gnar_item2': 'TFT_Item_UnstableConcoction',
 'TFT7_Anivia': 2,
 'TFT7_Anivia_item0': 'TFT_Item_ArchangelsStaff',
 'TFT7_Anivia_item1': 'TFT_Item_ArchangelsStaff',
 'TFT7_Anivia_item2': 'TFT_Item_ArchangelsStaff',
 'TFT7_Neeko': 2,
 'TFT7_Neeko_item0': 'TFT_Item_WarmogsArmor',
 'TFT7_Neeko_item1': 'TFT5_Item_RedemptionRadiant',
 'TFT7_Neeko_item2': 'TFT_Item_Quicksilver',
 'TFT7_DragonGreen': 2,
 'TFT7_DragonGreen_item0': 'TFT_Item_Bloodthirster',
 'TFT7_DragonGreen_item1': 'TFT_Item_GuinsoosRageblade',
 'TFT7_DragonGreen_item2': 'TFT_Item_GuinsoosRageblade'}

In [16]:
matches_league_df = pd.json_normalize(matches_array)

In [17]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Dragon,Set7_Evoker,Set7_Jade,Set7_Legend,Set7_Mystic,...,TFT7_Kayn_item1,TFT7_Kayn_item2,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_TahmKench_item1,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_TahmKench_item2,TFT7_Skarner_item2,TFT7_Senna_item2
0,KR_5993867535,7,TFT6_Augment_SalvageBin,TFT6_Augment_RadiantRelics,TFT6_Augment_ComponentGrabBag,1.0,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KR_5993867535,2,TFT6_Augment_SunfireBoard,TFT7_Augment_CavalierEmblem2,TFT7_Augment_DragonHorde,2.0,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KR_5993867535,4,TFT6_Augment_PortableForge,TFT6_Augment_BandOfThieves2,TFT6_Augment_CyberneticShell2,NaN,0.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KR_5993867535,5,TFT6_Augment_SunfireBoard,TFT6_Augment_MaxLevel10,TFT7_Augment_GuildEmblem,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KR_5993867535,3,TFT6_Augment_ThrillOfTheHunt2,TFT6_Augment_RadiantRelics,TFT6_Augment_PortableForge,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13259,KR_5991027338,3,TFT6_Augment_PandorasItems,TFT6_Augment_Electrocharge2,TFT6_Augment_Diversify3,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13260,KR_5991027338,5,TFT6_Augment_TinyTitans,TFT7_Augment_CannoneerEmblem,TFT6_Augment_HighEndShopping,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13261,KR_5991027338,8,TFT6_Augment_Weakspot,TFT7_Augment_UrfsGrabBag1,TFT7_Augment_UrfsGrabBag2,NaN,0.0,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13262,KR_5991027338,4,TFT6_Augment_LudensEcho1,TFT6_Augment_CelestialBlessing2,TFT6_Augment_GrandGambler,NaN,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Dragon,Set7_Evoker,Set7_Jade,Set7_Legend,Set7_Mystic,...,TFT7_Kayn_item1,TFT7_Kayn_item2,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_TahmKench_item1,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_TahmKench_item2,TFT7_Skarner_item2,TFT7_Senna_item2
0,KR_5993867535,7,TFT6_Augment_SalvageBin,TFT6_Augment_RadiantRelics,TFT6_Augment_ComponentGrabBag,1.0,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KR_5993867535,2,TFT6_Augment_SunfireBoard,TFT7_Augment_CavalierEmblem2,TFT7_Augment_DragonHorde,2.0,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KR_5993867535,4,TFT6_Augment_PortableForge,TFT6_Augment_BandOfThieves2,TFT6_Augment_CyberneticShell2,NaN,0.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KR_5993867535,5,TFT6_Augment_SunfireBoard,TFT6_Augment_MaxLevel10,TFT7_Augment_GuildEmblem,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KR_5993867535,3,TFT6_Augment_ThrillOfTheHunt2,TFT6_Augment_RadiantRelics,TFT6_Augment_PortableForge,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13259,KR_5991027338,3,TFT6_Augment_PandorasItems,TFT6_Augment_Electrocharge2,TFT6_Augment_Diversify3,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13260,KR_5991027338,5,TFT6_Augment_TinyTitans,TFT7_Augment_CannoneerEmblem,TFT6_Augment_HighEndShopping,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13261,KR_5991027338,8,TFT6_Augment_Weakspot,TFT7_Augment_UrfsGrabBag1,TFT7_Augment_UrfsGrabBag2,NaN,0.0,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13262,KR_5991027338,4,TFT6_Augment_LudensEcho1,TFT6_Augment_CelestialBlessing2,TFT6_Augment_GrandGambler,NaN,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)